# Phase 1: Aviation Risk Analysis

## 1. Introduction and Business Understanding

The objective of this project is to analyze aviation accident data from the NTSB to identify the lowest-risk aircraft for our company's new aviation division. Low risk is defined by a combination of low accident frequency and low fatality rates. The findings will be translated into three concrete business recommendations.

## 2. Data Understanding

In this section, we load the `AviationData.csv` dataset and perform initial inspections to understand its structure, data types, and the extent of missing values, which will guide our data cleaning strategy.

In [2]:
import pandas as pd
import numpy as np 

# Load the dataset
df = pd.read_csv('AviationData.csv', encoding='latin1')

# Display the first 5 rows to check structure
print(df.head())

         Event.Id Investigation.Type Accident.Number  Event.Date  \
0  20001218X45444           Accident      SEA87LA080  1948-10-24   
1  20001218X45447           Accident      LAX94LA336  1962-07-19   
2  20061025X01555           Accident      NYC07LA005  1974-08-30   
3  20001218X45448           Accident      LAX96LA321  1977-06-19   
4  20041105X01764           Accident      CHI79FA064  1979-08-02   

          Location        Country Latitude Longitude Airport.Code  \
0  MOOSE CREEK, ID  United States      NaN       NaN          NaN   
1   BRIDGEPORT, CA  United States      NaN       NaN          NaN   
2    Saltville, VA  United States  36.9222  -81.8781          NaN   
3       EUREKA, CA  United States      NaN       NaN          NaN   
4       Canton, OH  United States      NaN       NaN          NaN   

  Airport.Name  ... Purpose.of.flight Air.carrier Total.Fatal.Injuries  \
0          NaN  ...          Personal         NaN                  2.0   
1          NaN  ...         

C:\Users\Hp\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# To get a look at data types and non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

## 3. Data Preparation

My analysis requires accurate counts for injuries and a proper date format for time-series analysis. This section focuses on cleaning these key columns.

### 3.1 Handling Missing Injury Counts

For accident data, a missing count in injury columns (e.g., `Total.Fatal.Injuries`) often implies a count of zero. We will impute `NaN` values in these columns with 0, as dropping these records would eliminate valuable information about non-fatal accidents.

In [5]:
# Print the exact column names to determin what columns to work with
print(list(df.columns))

['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date', 'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 'Injury.Severity', 'Aircraft.damage', 'Aircraft.Category', 'Registration.Number', 'Make', 'Model', 'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description', 'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status', 'Publication.Date']


In [8]:
# List of columns related to injury counts
injury_columns = [
    'Total.Fatal.Injuries',
    'Total.Serious.Injuries',
    'Total.Minor.Injuries',
    'Total.Uninjured',
]

# Replacing NaN values with 0
df[injury_columns] = df[injury_columns].fillna(0)
print(df[injury_columns].isnull().sum())


Total.Fatal.Injuries      0
Total.Serious.Injuries    0
Total.Minor.Injuries      0
Total.Uninjured           0
dtype: int64


In [20]:
# Feature Engineering: Create column 'Total.Aboard'  by summing the individual counts to find the Fatality Rate metric.
df['Total.Aboard'] = (
    df['Total.Fatal.Injuries'] +
    df['Total.Serious.Injuries'] +
    df['Total.Minor.Injuries'] +
    df['Total.Uninjured']
)

df['Total.Aboard']

<bound method Series.min of 0        2.0
1        4.0
2        3.0
3        2.0
4        3.0
        ... 
88884    1.0
88885    0.0
88886    1.0
88887    0.0
88888    2.0
Name: Total.Aboard, Length: 88889, dtype: float64>

### 3.2 Converting and Filtering by Date

The `Event.Date` column must be converted to a proper datetime format to allow for filtering and trend analysis. We will also focus the analysis on the last few decades (e.g., since 1990) to ensure relevance for modern aircraft purchasing decisions.

In [5]:
# Convert the 'Event.Date' column to datetime objects
df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')

# Extract the year for filtering
df['Event.Year'] = df['Event.Date'].dt.year

# Filter the data to focus on events from 1990 onwards for modern risk assessment
# We use .copy() to ensure we are working on a clean subset of data.
df_recent = df[df['Event.Year'] >= 1990].copy()

# Save the cleaned DataFrame to CSV for verification and use in Tableau
df_recent.to_csv('AviationData_Cleaned_1990_Plus.csv', index=False)

NameError: name 'df' is not defined